In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

In [2]:
from liana.method.sp._Misty import MistyData
from liana.method.sp._misty_constructs import genericMistyData, lrMistyData

In [3]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

In [4]:
misty = genericMistyData(intra=adata, set_diag=True, bandwidth=10)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [5]:
misty(n_estimators=20, bypass_intra=False)

In [6]:
misty.uns['target_metrics']

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,intra,juxta,para
0,ECM,None,None,0.916356,0.934732,0.018376,0.769781,0.137327,0.092892
1,ligA,None,None,0.988767,0.989046,0.000279,0.966021,0.028719,0.005260
2,ligB,None,None,0.931236,0.945788,0.014552,0.790301,0.132289,0.077410
3,ligC,None,None,0.772155,0.867931,0.095776,0.581179,0.237349,0.181472
4,ligD,None,None,0.975195,0.976411,0.001216,0.928017,0.035186,0.036797
5,protE,None,None,0.996746,0.996751,0.000005,1.000000,0.000000,0.000000
6,protF,None,None,0.998839,0.998843,0.000004,0.994421,0.000000,0.005579
7,prodA,None,None,0.991433,0.991442,0.000008,1.000000,0.000000,0.000000
8,prodB,None,None,0.980717,0.980650,-0.000068,1.000000,0.000000,0.000000
9,prodC,None,None,0.954396,0.954739,0.000343,1.000000,0.000000,0.000000


In [7]:
adata = sc.pp.subsample(adata, n_obs=100, copy=True)

In [8]:
misty = genericMistyData(adata, bandwidth=20, add_juxta=True, set_diag=True, cutoff=0, coord_type="generic", delaunay=True)

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/mudata/_core/mudata.py:457: UserWarning: Cannot join columns with the same name because var_names are intersecting.


In [12]:
# Why is this thing so much slower?
misty(alphas=1, 
      bypass_intra=False,
      seed=42,
      n_estimators=100,
      keep_same_predictor=False,
      group_intra_by='cell_type',
      group_env_by='cell_type',
      )

/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
/home/dbdimitrov/anaconda3/envs/liana-py/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the spar

In [13]:
misty.uns['importances'].sort_values('value', ascending=False)

,target,predictor,intra_group,extra_group,view,value
322,prodB,ligB,A,A,intra,0.849296
332,prodB,ligB,A,B,intra,0.849296
393,prodC,ligC,B,B,intra,0.841559
383,prodC,ligC,B,A,intra,0.841559
97,ligB,prodB,A,B,intra,0.773047
...,...,...,...,...,...,...
339,prodB,prodD,A,B,intra,0.000000
429,prodD,prodC,B,A,intra,0.000000
317,prodA,prodB,B,B,intra,0.000000
428,prodD,prodB,B,A,intra,0.000000


In [15]:
misty.uns['target_metrics'] #NOTE: this is weird, why is the R2 not bound to 0-1???

,target,intra_group,env_group,intra.R2,multi.R2,gain.R2,intra,juxta,para
0,ECM,A,A,-1.733860,-1.041996,0.691864,0.737242,0.196139,0.066619
1,ECM,A,B,-1.733860,-1.135186,0.598674,0.840496,0.133151,0.026354
2,ECM,B,A,-4.860504,-3.698673,1.161831,1.000000,0.000000,0.000000
3,ECM,B,B,-4.860504,-4.501236,0.359268,0.885328,0.026307,0.088365
4,ligA,A,A,0.273891,0.286273,0.012382,0.777551,0.053388,0.169061
5,ligA,A,B,0.273891,0.256427,-0.017464,0.844439,0.155561,0.000000
6,ligA,B,A,0.603905,0.598658,-0.005248,1.000000,0.000000,0.000000
7,ligA,B,B,0.603905,0.595976,-0.007929,0.924492,0.059436,0.016072
8,ligB,A,A,-0.994064,-1.009762,-0.015698,0.966081,0.000000,0.033919
9,ligB,A,B,-0.994064,-0.981054,0.013010,1.000000,0.000000,0.000000


In [ ]:
misty

In [ ]:
misty.uns['target_metrics']

In [ ]:
misty.uns['importances']

lrMistyData

In [ ]:
from liana.testing._sample_anndata import generate_toy_spatial

In [ ]:
adata = generate_toy_spatial()

In [ ]:
adata.raw.X.tocsr()

In [ ]:
lr_misty = lrMistyData(adata=adata, set_diag=True, bandwidth=10, use_raw=True)

In [ ]:
lr_misty(bypass_intra=True)

In [ ]:
lr_misty.uns['target_metrics']

In [ ]:
lr_misty.uns['importances'].sort_values('value', ascending=False)

In [ ]:
from liana.resource import select_resource
from liana.method._pipe_utils import prep_check_adata
from liana.method.sp._lr_basis import _add_complexes_to_var
from liana.method.sp._misty_constructs import _make_view
from liana.method.sp._spatial_pipe import spatial_neighbors

In [ ]:
def lrMistyData(adata, 
                resource_name='consensus', 
                resource=None,
                nz_threshold=0.1,
                use_raw = False,
                layer = None,
                spatial_key='spatial',
                bandwidth = 100,
                kernel = 'misty_rbf',
                set_diag = True,
                cutoff = 0.1,
                zoi = 0,
                verbose = False
                ):
    
    if resource is None:
        resource = select_resource(resource_name.lower())
        
    adata = prep_check_adata(adata=adata,
                        use_raw=use_raw,
                        layer=layer,
                        verbose=verbose,
                        groupby=None,
                        min_cells=None,
                        obsm = {spatial_key: adata.obsm[spatial_key]}
                        )
    
    adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )
    
    views = dict()
    intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)
    views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))
    
    connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

    extra = _make_view(adata=adata[:,resource['ligand'].unique()],
                       spatial_key=spatial_key, nz_threshold=nz_threshold,
                       connecitivity=connectivity,
                       obs=pd.DataFrame(index=adata.obs.index))
    views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))
    
    return MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

In [ ]:
use_raw = False
layer = None
verbose = True
resource = None
resource_name = 'consensus'
nz_threshold = 0.1

In [ ]:
spatial_key = 'spatial'
bandwidth = 100
kernel = 'misty_rbf'
set_diag = True
cutoff = 0.1
zoi = 0

In [ ]:
if resource is None:
    resource = select_resource(resource_name.lower())

In [ ]:
views = dict()

In [ ]:
adata = prep_check_adata(adata=adata,
                         use_raw=use_raw,
                         layer=layer,
                         verbose=verbose,
                         groupby=None,
                         min_cells=None,
                         obsm = {spatial_key: adata.obsm[spatial_key]}
                         )

In [ ]:
adata = _add_complexes_to_var(adata,
                              np.union1d(resource['receptor'].astype(str),
                                         resource['ligand'].astype(str)
                                        )
                              )

In [ ]:
adata

In [ ]:
# filter_resource
resource = resource[(np.isin(resource.ligand, adata.var_names)) &
                    (np.isin(resource.receptor, adata.var_names))]

In [ ]:
intra = _make_view(adata=adata[:, resource['receptor'].unique()],
                   nz_threshold=nz_threshold, obs=adata.obs)

views['intra'] = _add_complexes_to_var(intra, np.unique(resource['receptor']))

In [ ]:
connectivity = spatial_neighbors(adata=adata, spatial_key=spatial_key,
                                 bandwidth=bandwidth, kernel=kernel,
                                 set_diag=set_diag, cutoff=cutoff,
                                 zoi=zoi, inplace=False)

extra = _make_view(adata=adata[:,resource['ligand'].unique()], spatial_key=spatial_key,
                   nz_threshold=nz_threshold, connecitivity=connectivity, obs=pd.DataFrame(index=adata.obs.index))
views['extra'] = _add_complexes_to_var(extra, np.unique(resource['ligand']))

In [ ]:
from mudata import MuData

In [ ]:
lr_misty = MistyData(data=views, obs=intra.obs, spatial_key=spatial_key)

In [ ]:
lr_misty(bypass_intra=False)

In [ ]:
lr_misty.uns['target_metrics'].sort_values('multi.R2')

In [ ]:
lr_misty.uns['importances'].sort_values('value', ascending=False).head(20)